In [57]:
import pandas as pd
import numpy as np
import re
import turbodbc
import os

In [97]:
def create_connection_string_turbo(server, database):
    options = turbodbc.make_options(prefer_unicode=True)
    constr = 'Driver={ODBC Driver 13 for SQL Server};Server=' + \
        server + ';Database=' + database + ';Trusted_Connection=yes;'
    con = turbodbc.connect(connection_string=constr, turbodbc_options=options)
    return con

In [98]:
def sql_to_pandas(connection, query, *args, **kwargs):
    df = pd.read_sql(query, connection, *args, **kwargs)
    return df

In [62]:
company = 'TeamSaniDusch'

In [63]:
files = []
for i in os.listdir("../Archiv"):
    for j in os.listdir(os.path.join("../Archiv", i)):
        if re.match(company+".*", j):
            files += [os.path.join("../Archiv", i, j)]

In [113]:
df = pd.DataFrame()
for i in files:
    df_temp = pd.read_csv(i, sep=";", dtype=str)
    df_temp['Timestamp'] = i.split("\\")[1]
    df = pd.concat([df, df_temp], axis=0)

In [114]:
df.fillna('', inplace=True)
df['FarbId'] = df['FarbId'].replace('', '000')

In [115]:
df['UID'] = df[['ArtikelId', 'FarbId', 'AusführungsId']].apply(lambda x: ''.join(x), axis=1)

In [116]:
df = df[['UID', 'Timestamp', 'Preis_Pos']].copy()

In [117]:
df_pivot = df[df['Timestamp'].apply(lambda x: x in ['2017-11-27', '2018-01-15'])].pivot(index="UID", columns="Timestamp", values="Preis_Pos")
df_pivot = df_pivot.astype(np.float)
df_pivot['Diff'] = df_pivot.iloc[:,1] -  df_pivot.iloc[:,0]
df_pivot['Diff%'] = df_pivot['Diff'] / df_pivot.iloc[:,0]

In [122]:
query = """SELECT Artikelnummer,
                [Artikel Beschreibung],
                SGVSB,
                Bruttoverkauf,
                Umsatz = Sales
            FROM (
            SELECT Artikelnummer = x.[Artikel]
                ,[Artikel Beschreibung] = [Artikel Beschreibung]
                ,[SGVSB]
                ,[Bruttoverkauf]
            FROM [BDL].[DES].[Artikelstamm] x
            LEFT OUTER JOIN (SELECT Artikelnummer = Artikel, [Brutto Einkaufspreis] = max([Brutto Einkaufspreis]), Bruttoverkauf = max(Bruttoverkauf), Standardkosten = max(Standardkosten), [Netto Einkaufspreis Pro LE] = max([Netto Einkaufspreis Pro LE])
                            FROM [BDL].[DES].[Artikel Sparten Konditionen]
                            GROUP BY Artikel ) y on y.Artikelnummer = x.Artikel
            where SGVSB > '' and Lieferant <> '00008537'
            ) x 
            left join (SELECT iditemOrigin, Sales = Sum(Sales)
                        from InfoPool.Fact.Sales where year = 2017 group by iditemorigin ) y on y.idItemOrigin = x.Artikelnummer collate SQL_Latin1_General_CP1_CI_AI"""

In [123]:
con = create_connection_string_turbo("CRHBUSADWH51", 'BDL')

In [124]:
df_meta = sql_to_pandas(con, query)
df_meta = df_meta.set_index('SGVSB')

In [128]:
df_pivot = df_pivot.join(df_meta).sort_values('Umsatz', ascending = False)

In [129]:
df_pivot.to_clipboard()